In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from sklearn import preprocessing    
from ast import literal_eval

In [5]:
def writetrainann(file):
    maxl = 0
    columns = ['name', 'label']
    names = []
    labels = []
    categories = []
    with open(file, 'r') as f:
        ann = json.loads(f.read())
        for key in ann.keys():
            img_dict = ann[key]
            img_name = img_dict['name']
            cat_list = img_dict['category']
            maxl = max(maxl, len(cat_list))
            categories.extend(cat_list)
            
            for cat in cat_list:
                names.append(img_name)
                labels.append(cat)
                
        data = pd.DataFrame(list(zip(names, labels)), columns=columns)
        print('Maximum categories per image', maxl)
        print('Total number of categories', len(set(categories)))
        print('All categories', set(categories))
        data.to_csv('../Data/train_ann.csv', index = False)                   

In [9]:
ann_dir = "../../../Data/annotation"
train_fname = 'i2l_trainset_annotation.json'
train_fname = os.path.join(ann_dir, train_fname)
train_data = writetrainann(train_fname)

Maximum categories per image 9
Total number of categories 79
All categories {'frisbee', 'microwave', 'chair', 'orange', 'cup', 'keyboard', 'scissors', 'vase', 'hot dog', 'cell phone', 'stop sign', 'car', 'tv', 'laptop', 'skis', 'bottle', 'sink', 'broccoli', 'fork', 'book', 'horse', 'sports ball', 'tie', 'skateboard', 'truck', 'toothbrush', 'sandwich', 'cake', 'bowl', 'bicycle', 'dog', 'spoon', 'bird', 'apple', 'tennis racket', 'bear', 'suitcase', 'dining table', 'parking meter', 'couch', 'elephant', 'backpack', 'banana', 'remote', 'umbrella', 'sheep', 'clock', 'giraffe', 'wine glass', 'knife', 'bus', 'baseball bat', 'handbag', 'bed', 'cat', 'baseball glove', 'traffic light', 'kite', 'carrot', 'fire hydrant', 'hair drier', 'train', 'potted plant', 'teddy bear', 'donut', 'toaster', 'pizza', 'bench', 'surfboard', 'toilet', 'mouse', 'motorcycle', 'airplane', 'cow', 'zebra', 'refrigerator', 'snowboard', 'boat', 'oven'}


In [12]:
# Checking training examples per-class. Dataset Stats

train_data = pd.read_csv('../Data/train_ann.csv')
train_groups = train_data.groupby(['label'])
count_df = train_groups.agg('count')

print('Max Freq category:', train_data['label'].value_counts().idxmax())
print('Min Freq category:',train_data['label'].value_counts().idxmin())
print('Mean Freq:',train_data['label'].value_counts().mean())
print(count_df)

Max Freq category: chair
Min Freq category: bear
Mean Freq: 599.620253164557
                name
label               
airplane         141
apple            290
backpack        1394
banana           293
baseball bat     590
baseball glove   710
bear              44
bed              197
bench            925
bicycle          615
bird             397
boat             275
book            1164
bottle          2190
bowl            1220
broccoli         207
bus              400
cake             263
car             2139
carrot           247
cat              277
cell phone      1277
chair           2243
clock           1145
couch            568
cow               99
cup             2241
dining table     762
dog              583
donut            135
...              ...
pizza            127
potted plant     957
refrigerator     570
remote           646
sandwich         200
scissors         191
sheep             56
sink            1073
skateboard       463
skis             364
snowboard        248

In [ ]:
# Encoding labels (0-78)

train_data = pd.read_csv('../Data/train_ann.csv')
le = preprocessing.LabelEncoder()
train_data['label_enc'] = le.fit_transform(train_data['label'])
train_data.to_csv('../Data/train_ann_encoded.csv', index = False)   

label_enc_dict = pd.Series(train_data.label_enc.values,index=train_data.label).to_dict()
print(label_enc_dict)

label_enc_json = json.dumps(label_enc_dict)
f = open("../Data/labels.json","w")
f.write(label_enc_json)
f.close()

In [17]:
def writeann_multilabel(file):
    maxl = 0
    columns = ['name', 'label']
    names = []
    labels = []
    with open(file, 'r') as f:
        ann = json.loads(f.read())
        for key in ann.keys():
            img_dict = ann[key]
            img_name = img_dict['name']
            cat_list = img_dict['category']
            maxl = max(maxl, len(cat_list))
            category = " ".join(cat_list)
            names.append(img_name)
            labels.append(category)
        
        data = pd.DataFrame(list(zip(names, labels)), columns=columns)
        data.to_csv('../Data/train_ml_ann.csv', index = False)                   

In [18]:
ann_dir = "../../../Data/annotation"
train_fname = 'i2l_trainset_annotation.json'
train_fname = os.path.join(ann_dir, train_fname)
writeann_multilabel(train_fname)

                    name                   label
0  000000016164_left.png                     dog
1  000000471175_left.png                     dog
2  000000026654_left.png                     dog
3  000000158497_left.png  spoon refrigerator dog
4  000000390348_left.png                     dog
5  000000369190_left.png                     dog
6  000000151988_left.png              bottle dog
7  000000307993_left.png                book dog
8  000000007125_left.png                     dog
9  000000346965_left.png                     dog


In [23]:
def writeann_multilabel_enc(file):
    
    with open("../Data/labels.json","r") as f:
        encodings = json.loads(f.read())
    
    maxl = 0
    columns = ['name', 'label']
    names = []
    labels = []
    with open(file, 'r') as f:
        ann = json.loads(f.read())
        for key in ann.keys():
            img_dict = ann[key]
            img_name = img_dict['name']
            cat_list = img_dict['category']
            maxl = max(maxl, len(cat_list))
            cat_labels = []
            for cat in cat_list:
                cat_labels.append(encodings[cat])
            
            category = cat_labels
            names.append(img_name)
            labels.append(category)
        
        data = pd.DataFrame(list(zip(names, labels)), columns=columns)
        print(data.head(5))
        data.to_csv('../Data/train_ml_ann_encoded.csv', index = False) 

In [24]:
# Applying label encoding on trainig_multilabel data

ann_dir = "../../../Data/annotation"
train_fname = 'i2l_trainset_annotation.json'
train_fname = os.path.join(ann_dir, train_fname)
writeann_multilabel_enc(train_fname)

                    name         label
0  000000016164_left.png          [28]
1  000000471175_left.png          [28]
2  000000026654_left.png          [28]
3  000000158497_left.png  [60, 51, 28]
4  000000390348_left.png          [28]


In [54]:
def writevalann(file):
    maxl = 0
    columns = ['name', 'label', 'WC1', 'WC2', 'WC3', 'WC4']
    names = []
    labels = []
    WC1 = []
    WC2 = []
    WC3 = []
    WC4 = []
    with open(file, 'r') as f:
        ann = json.loads(f.read())
        for key in ann.keys():
            img_dict = ann[key]
            img_name = img_dict['name']
            correct_cand = img_dict['correct_candidate']
            wrong_cands = img_dict['wrong_candidate'] # four element always
            names.append(img_name)
            labels.append(correct_cand[0]) # only one element always
            WC1.append(wrong_cands[0])
            WC2.append(wrong_cands[1])
            WC3.append(wrong_cands[2])
            WC4.append(wrong_cands[3])
    
        data = pd.DataFrame(list(zip(names, labels, WC1, WC2, WC3, WC4)), columns=columns)
        data.to_csv('../Data/val_ann.csv', index = False)

In [55]:
ann_dir = "../../../Data/annotation"
val_fname = 'i2l_valset_annotation.json'
val_fname = os.path.join(ann_dir, val_fname)
writevalann(val_fname)

                    name          label       WC1         WC2            WC3  \
0  000000116358_left.png    sports ball     sheep       apple       backpack   
1  000000050179_left.png  tennis racket  airplane    elephant            tie   
2  000000523571_left.png   dining table       car     frisbee     cell phone   
3  000000531798_left.png           bowl      cake  wine glass     teddy bear   
4  000000531798_left.png         banana    toilet         cup  parking meter   

     WC4  
0  mouse  
1   kite  
2  pizza  
3  clock  
4   vase  


In [56]:
def writetestann(file):
    maxl = 0
    columns = ['name', 'label', 'WC1', 'WC2', 'WC3', 'WC4']
    names = []
    labels = []
    WC1 = []
    WC2 = []
    WC3 = []
    WC4 = []
    with open(file, 'r') as f:
        ann = json.loads(f.read())
        for key in ann.keys():
            img_dict = ann[key]
            img_name = img_dict['name']
            correct_cand = img_dict['correct_candidate']
            wrong_cands = img_dict['wrong_candidate'] # four element always
            names.append(img_name)
            labels.append(correct_cand[0]) # only one element always
            WC1.append(wrong_cands[0])
            WC2.append(wrong_cands[1])
            WC3.append(wrong_cands[2])
            WC4.append(wrong_cands[3])
    
        data = pd.DataFrame(list(zip(names, labels, WC1, WC2, WC3, WC4)), columns=columns)
        data.to_csv('../Data/test_ann.csv', index = False)

In [57]:
ann_dir = "../../../Data/annotation"
test_fname = 'i2l_testset_annotation.json'
test_fname = os.path.join(ann_dir, test_fname)
writetestann(test_fname)

In [16]:
# Applying label encoding on test and validation data (using labels applied for training)
val_data = pd.read_csv('../Data/val_ann.csv')
test_data = pd.read_csv('../Data/test_ann.csv')

with open("../Data/labels.json","r") as f:
    encodings = json.loads(f.read())
    for col in val_data.columns[1:]:
        val_data[col].replace(encodings, inplace=True)
        
    for col in test_data.columns[1:]:
        test_data[col].replace(encodings, inplace=True)

val_data.to_csv('../Data/val_ann_encoded.csv', index = False)  
test_data.to_csv('../Data/test_ann_encoded.csv', index = False)  

In [3]:
def write_hybrid_left_labels(file):
    
    columns = ['name', 'label']
    names = []
    labels = []
    with open(file, 'r') as f:
        ann = json.loads(f.read())
        for key in ann.keys():
            img_dict = ann[key]
            img_name = img_dict['name']
            cat_list = img_dict['category_symm']
    
            cat_labels = []
            for cat in cat_list:
                cat_labels.append(cat)
            
            category = cat_labels
            names.append(img_name)
            labels.append(category)
        
        data = pd.DataFrame(list(zip(names, labels)), columns=columns)
        print(data.head(5))
        data.to_csv('../Data/hybrid_train_ann_left_labels.csv', index = False) 

In [4]:
# Store left side image labels as list

ann_dir = "../../../Data/annotation"
hybrid_fname = 'hybrid_trainset_annotation.json'
hybrid_fname = os.path.join(ann_dir, train_fname)
# write_hybrid_left_labels(hybrid_fname)

NameError: name 'os' is not defined

In [14]:
def getCorrMatrix(labels, encodings):
    C = len(encodings.keys())

    correlation_mat = np.zeros((C,C))
    nrows = labels.shape[0]
    print('classes: ',C)
    print('images: ',nrows)
    
    nums = np.zeros(C)

    for r in range(nrows):
        list_label = literal_eval(labels.iloc[r])
        n = len(list_label)
        for a in range(n):
            i = encodings[list_label[a]]
            nums[i] += 1
            for b in range(a + 1, n):
                j = encodings[list_label[b]]
                correlation_mat[i][j] += 1
                correlation_mat[j][i] += 1
                
    return correlation_mat, nums

In [15]:
# Generating correlation matrix for ML-GCN

hybrid_csv = pd.read_csv('../Data/hybrid_train_ann_left_labels.csv')
labels = hybrid_csv['label']
with open("../Data/labels.json","r") as f:
        encodings = json.loads(f.read())

adj, nums = getCorrMatrix(labels, encodings)

corrMat = {}
corrMat['adj'] = adj
corrMat['nums'] = nums

with open('../Data/baseline_left_labels.pkl', 'wb') as f:
    pickle.dump(corrMat, f)

classes:  79
images:  32000


In [16]:
# with open('../Data/baseline_left_labels.pkl', 'rb') as f:
#     corr = pickle.load(f)

# print(corr['nums'])
# print(len(corr['adj']), len(corr['nums']))

[  87.  280.  877.  278.  398.  487.    9.  144.  690.  544.  238.  246.
 1296. 2117. 1510.  224.  378.  329. 2246.  310.  165.  735. 3305.  487.
  730.  119. 2302.  882.  280.  175.   60.  129.  737.  131.   49.   21.
 1286.  172.  118.  340.  211.  860.  563.  325.  377.  221.  262.  505.
   78.  256.  948.  457.  689.  278.   97.  114.  744.  196.  133.   53.
  741.  571.  134.  297.   93.  131.  413.  456.   48.  273.   99.  756.
  102.  658.  857.  518.  664.  686.   45.]
79 79


In [19]:
vg_ml_train = pd.read_csv("../Data/visual_genome/visual_genome_train_ann_ml.csv")
vg_ml_val = pd.read_csv("../Data/visual_genome/visual_genome_val_ann_ml.csv")
print(vg_ml_train.shape, vg__ml_val.shape)
print(vg_ml_train.columns)

(59601, 3) (14815, 3)
Index(['image_id', 'object_names', 'object_ids'], dtype='object')


In [2]:
def split_labels(df):
    sample_list = []  # append one row at a time to this
    
    nrows = df.shape[0]
    count = 0
    for r in range(nrows):  # generate n rows for each row in original table
        list_label = literal_eval(df['object_ids'].iloc[r])
        list_objects = literal_eval(df['object_names'].iloc[r])
        img_id = df['image_id'].iloc[r]
        
        n = len(list_label)
        count += n
        for i in range(n):
            new_row = []
            new_row.append(img_id)
            new_row.append([list_objects[i]]) # add as a list of size = 1
            new_row.append([list_label[i]])
            sample_list.append(new_row)
    
    print('count', count)
    print(len(sample_list), len(sample_list[0]))
    final_columns = ['image_id', 'object_names', 'object_ids']
    final_df = pd.DataFrame(sample_list,columns=final_columns)
    print(final_df.shape)
    return final_df

In [21]:
train_df = split_labels(vg_ml_train)
print(train_df.image_id.nunique())
train_df.to_csv('../Data/visual_genome/visual_genome_train_ann.csv', index = False)

val_df = split_labels(vg_ml_val)
print(val_df.image_id.nunique())
val_df.to_csv('../Data/visual_genome/visual_genome_val_ann.csv', index = False)



count 111622
111622 3
(111622, 3)
59601
count 27929
27929 3
(27929, 3)
14815


In [3]:
vg_ml_df = pd.read_csv('../data/visual_genome/visual_genome_ann.csv')

In [4]:
vg_df = split_labels(vg_ml_df)
print(vg_df.image_id.nunique())
vg_df.to_csv('../data/visual_genome/visual_genome_ann.csv', index=False)

count 174424
174424 3
(174424, 3)
93016


In [5]:
vg_df.head()

,image_id,object_names,object_ids
0,1,[parking meter],[12]
1,1,[clock],[74]
2,1,[car],[2]
3,2,[backpack],[24]
4,2,[car],[2]


In [3]:
vg_temp = pd.read_csv('../data/visual_genome/visual_genome_gcn.csv')

In [4]:
vg_temp = split_labels(vg_temp)

count 174424
174424 3
(174424, 3)


In [6]:
for index, row in vg_temp.iterrows():
    if index % 1000 == 0:
        print(index)
    
    if vg_temp.iloc[index, 2] == 79:
        vg_temp.iloc[index, 2] = 77
    elif vg_temp.iloc[index, 2] > 32:
        vg_temp.iloc[index, 2] -= 1

0


TypeError: '>' not supported between instances of 'list' and 'int'